# Image Augmentation with Bounding Box Tracking

This notebook augments object images by blending them into background scenes with rotation and transparency, while tracking bounding boxes.

## Imports and Configuration

In [1]:

import os
import cv2
import random
import numpy as np
import pandas as pd
from typing import List, Tuple

# Configuration
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png"}
OUTPUT_DIR = "augmented_images"
ANNOTATIONS_FILE = "annotations.xlsx"


## Utility Functions

Functions for loading images, ensuring alpha channels, and basic image operations.

In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from typing import List, Tuple
def load_image(path: str) -> np.ndarray:
    """Loads an image with support for alpha channel (transparency)."""
    return cv2.imread(path, cv2.IMREAD_UNCHANGED)

def list_image_files(directory: str, extensions={".jpg", ".jpeg", ".png"}) -> List[str]:
    """List all image files in a directory with given extensions."""
    return [os.path.join(directory, f) for f in os.listdir(directory)
            if os.path.splitext(f)[1].lower() in extensions]
def add_alpha_if_missing(image: np.ndarray) -> np.ndarray:
    """Ensure image has 4 channels (RGBA)."""
    if image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
        image[:, :, 3] = 255
    return image

def rotate_image(image: np.ndarray, angle: float) -> np.ndarray:
    """Rotate the image around its center."""
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    return cv2.warpAffine(image, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderValue=(0, 0, 0, 0))

def crop_and_prepare_sheep(image: np.ndarray, bbox: Tuple[int, int, int, int], max_size: int = 60) -> np.ndarray:
    """
    Crop the sheep from the image, ensure alpha channel, and apply random rotation.
    """
    x, y, w, h = bbox
    sheep = image[y:y+h, x:x+w]
    sheep = add_alpha_if_missing(sheep)

    angle = random.uniform(0, 360)
    return rotate_image(sheep, angle)




## Augmentation Functions

Functions for cropping, rotating, and blending object images (e.g., sheep) onto backgrounds.

In [3]:
def match_background_texture(sheep_rgb: np.ndarray, background_roi: np.ndarray) -> np.ndarray:
    """
    Adjust brightness to match the background ROI.
    """
    mean_bg = np.mean(background_roi)
    mean_sheep = np.mean(sheep_rgb)
    alpha = mean_bg / max(mean_sheep, 1)
    return cv2.convertScaleAbs(sheep_rgb, alpha=alpha)


def paste_sheep(background: np.ndarray, sheep: np.ndarray) -> np.ndarray:
    """
    Paste one sheep using feathered Poisson blending and localized brightness adjustment.
    """
    if sheep.shape[2] != 4:
        return background

    mask = cv2.threshold(sheep[:, :, 3], 1, 255, cv2.THRESH_BINARY)[1]
    sheep_rgb = sheep[:, :, :3]

    bg_h, bg_w = background.shape[:2]
    sh_h, sh_w = sheep_rgb.shape[:2]

    if sh_h >= bg_h or sh_w >= bg_w:
        return background

    x = random.randint(0, bg_w - sh_w)
    y = random.randint(0, bg_h - sh_h)
    center = (x + sh_w // 2, y + sh_h // 2)

    # ✅ Feather mask
    mask = cv2.GaussianBlur(mask, (3, 3), 0)

    # ✅ Brightness match using ROI
    roi = background[y:y+sh_h, x:x+sh_w]
    sheep_rgb = match_background_texture(sheep_rgb, roi)

    try:
        return cv2.seamlessClone(sheep_rgb, background, mask, center, cv2.MIXED_CLONE)
    except:
        return background


def generate_augmented_images_from_annotations(
    images_folder: str,
    annotations_path: str,
    output_folder: str,
    amount: int = 5,
    sheep_min: int = 5,
    sheep_max: int = 10
):
    """
    Generate synthetic images using annotated sheep images and placing them on random backgrounds.
    
    Args:
        images_folder (str): Path to the folder with all images.
        annotations_path (str): Path to the Excel file containing bounding box annotations.
        output_folder (str): Folder where generated images will be saved.
        amount (int): Number of images to generate.
        sheep_min (int): Minimum number of sheep per image.
        sheep_max (int): Maximum number of sheep per image.
    """
    os.makedirs(output_folder, exist_ok=True)

    df = pd.read_excel(annotations_path)
    grouped = df.groupby("image_name")

    sheep_image_names = list(grouped.groups.keys())

    bounding_boxes = [
        [
            (row["bbox_x"], row["bbox_y"], row["bbox_width"], row["bbox_height"])
            for _, row in group.iterrows()
        ]
        for _, group in grouped
    ]

    all_images = list_image_files(images_folder)
    sheep_images = [os.path.join(images_folder, name) for name in sheep_image_names if os.path.exists(os.path.join(images_folder, name))]
    background_images = [img for img in all_images if os.path.basename(img) not in sheep_image_names]

    for i in range(amount):
        background = load_image(random.choice(background_images))
        result = background.copy()

        num_sheep = random.randint(sheep_min, sheep_max)

        for _ in range(num_sheep):
            idx = random.randint(0, len(sheep_images) - 1)
            boxes = bounding_boxes[idx]
            if not boxes:
                continue

            sheep_img = load_image(sheep_images[idx])
            bbox = random.choice(boxes)
            sheep = crop_and_prepare_sheep(sheep_img, bbox)
            result = paste_sheep(result, sheep)

        output_path = os.path.join(output_folder, f"augmented_{i}.jpg")
        cv2.imwrite(output_path, result)
        print(f"Saved: {output_path}")


data_folder = "../dataset"
images_folder = f"{data_folder}/images"
annotations_path = f"{data_folder}/annotations.xlsx"
output_folder = f"{data_folder}/augmented"

generate_augmented_images_from_annotations(
    images_folder=images_folder,
    annotations_path=annotations_path,
    output_folder=output_folder,
    amount=10,
    sheep_min=3,     
    sheep_max=8   
)

Saved: ../dataset/augmented\augmented_0.jpg
Saved: ../dataset/augmented\augmented_1.jpg
Saved: ../dataset/augmented\augmented_2.jpg
Saved: ../dataset/augmented\augmented_3.jpg
Saved: ../dataset/augmented\augmented_4.jpg
Saved: ../dataset/augmented\augmented_5.jpg
Saved: ../dataset/augmented\augmented_6.jpg
Saved: ../dataset/augmented\augmented_7.jpg
Saved: ../dataset/augmented\augmented_8.jpg
Saved: ../dataset/augmented\augmented_9.jpg


## Main Augmentation Process

Loop through background and object images, perform augmentation, and collect bounding box data.

In [4]:
# The main augmentation loop would be here, invoking above functions appropriately.

## Export Annotations

Save the collected bounding box annotations to an Excel file.

In [5]:

def export_annotations(bbox_data: list, output_path: str) -> None:
    """Save bounding box annotations to an Excel file."""
    df = pd.DataFrame(bbox_data, columns=["filename", "class", "x_min", "y_min", "x_max", "y_max"])
    df.to_excel(output_path, index=False)

# Example usage after processing
# export_annotations(all_bboxes, os.path.join(OUTPUT_DIR, ANNOTATIONS_FILE))


## Refactored Augmentation Functions

In [6]:

def calculate_paste_position(bg_shape, sheep_shape) -> Tuple[int, int]:
    """Calculate a random valid paste position on the background."""
    bg_h, bg_w = bg_shape[:2]
    sh_h, sh_w = sheep_shape[:2]
    max_x = max(bg_w - sh_w, 1)
    max_y = max(bg_h - sh_h, 1)
    return random.randint(0, max_x), random.randint(0, max_y)

def blend_images(background, foreground, x_offset, y_offset) -> np.ndarray:
    """Alpha blend the foreground (sheep) onto the background."""
    bg = background.copy()
    h, w = foreground.shape[:2]
    alpha_s = foreground[:, :, 3] / 255.0
    alpha_b = 1.0 - alpha_s

    for c in range(3):
        bg[y_offset:y_offset+h, x_offset:x_offset+w, c] = (
            alpha_s * foreground[:, :, c] +
            alpha_b * bg[y_offset:y_offset+h, x_offset:x_offset+w, c]
        )
    return bg

def compute_bounding_box(x_offset, y_offset, sheep_shape) -> Tuple[int, int, int, int]:
    """Compute bounding box for the pasted object."""
    sh_h, sh_w = sheep_shape[:2]
    return x_offset, y_offset, x_offset + sh_w, y_offset + sh_h

def paste_sheep(background: np.ndarray, sheep: np.ndarray) -> Tuple[np.ndarray, Tuple[int, int, int, int]]:
    """Paste sheep image onto background with blending and return new image and bbox."""
    x_offset, y_offset = calculate_paste_position(background.shape, sheep.shape)
    blended_image = blend_images(background, sheep, x_offset, y_offset)
    bbox = compute_bounding_box(x_offset, y_offset, sheep.shape)
    return blended_image, bbox

def generate_augmented_images_from_annotations(annotation_df: pd.DataFrame, backgrounds: List[str], sheep_images: str, output_dir: str) -> list:
    """Main function to generate augmented images and collect bounding boxes."""
    all_bboxes = []
    os.makedirs(output_dir, exist_ok=True)
    sheep_files = list_image_files(sheep_images)
    
    for idx, row in annotation_df.iterrows():
        bg_path = random.choice(backgrounds)
        bg_img = add_alpha_if_missing(load_image(bg_path))
        sheep_path = random.choice(sheep_files)
        sheep_img = add_alpha_if_missing(load_image(sheep_path))
        
        bbox = (10, 10, sheep_img.shape[1] - 20, sheep_img.shape[0] - 20)
        prepared_sheep = crop_and_prepare_sheep(sheep_img, bbox)
        
        result_img, new_bbox = paste_sheep(bg_img, prepared_sheep)
        filename = f"aug_{idx}.png"
        cv2.imwrite(os.path.join(output_dir, filename), result_img)
        
        all_bboxes.append([filename, "sheep", *new_bbox])
    
    return all_bboxes
